In [1]:
import torch
import torch.optim
import pandas as pd
import os

import nnts
import nnts.data
from nnts import utils
import nnts.torch.models
import nnts.torch.trainers
import nnts.metrics
import nnts.torch.datasets
import nnts.torch.utils
import nnts.loggers
from nnts import datasets
torch.set_printoptions(precision=8, sci_mode=False)
%load_ext autoreload
%autoreload 2

In [2]:
data_path = "data"
model_name = ""
dataset_name = "ETTh2"
results_path = "nb-results"
metadata_path = "informer.json"

metadata = datasets.load_metadata(dataset_name, path=metadata_path)
datafile_path = os.path.join(data_path, metadata.filename)
PATH = os.path.join(results_path, model_name, metadata.dataset)
df = pd.read_csv(datafile_path)
utils.makedirs_if_not_exists(PATH)

In [3]:
params = utils.Hyperparams(
    optimizer=torch.optim.Adam,
    loss_fn=torch.nn.L1Loss(),
    batch_size=32,
    batches_per_epoch=50,
    training_method=utils.TrainingMethod.DMS,
    model_file_path="logs",
    epochs=100,
    scheduler=utils.Scheduler.REDUCE_LR_ON_PLATEAU,
)

In [ ]:
#ONLY RUN FOR UNIVARIATE
df = pd.melt(df, id_vars=["date"], value_vars=["HUFL", "HULL", "MUFL", "MULL", "LUFL", "LULL", "OT"], var_name="unique_id", value_name="y")
df = df.rename({"date": "ds"}, axis="columns")
df.head()

,ds,unique_id,y
0,2016-07-01 00:00:00,HUFL,41.130001
1,2016-07-01 01:00:00,HUFL,37.528000
2,2016-07-01 02:00:00,HUFL,37.946999
3,2016-07-01 03:00:00,HUFL,38.952000
4,2016-07-01 04:00:00,HUFL,38.113998


In [4]:
df.tail()

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
17415,2018-06-26 15:00:00,39.202999,11.392,49.644001,11.929,-10.331,-1.258,47.084999
17416,2018-06-26 16:00:00,38.113998,10.974,48.759998,11.366,-10.331,-1.290,48.183498
17417,2018-06-26 17:00:00,39.622002,10.974,50.609001,11.661,-11.557,-1.418,48.183498
17418,2018-06-26 18:00:00,43.643002,13.403,54.737000,13.778,-10.299,-1.418,46.865501
17419,2018-06-26 19:00:00,38.868000,10.052,49.859001,10.669,-11.525,-1.418,45.986500


In [5]:
df = df.rename({"OT": "y", "date": "ds"}, axis="columns")
df["unique_id"] = "T1"

In [6]:
df.columns

Index(['ds', 'HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL', 'y', 'unique_id'], dtype='object')

In [7]:
# split lengths as per informer
trn_length = int(24 * 365.25)
val_test_length = int(24 * 365.25 * (4 / 12))
split_data = datasets.split_test_val_train(
    df, trn_length, val_test_length, val_test_length
)

In [10]:
nnts.torch.utils.seed_everything(42)
dataset_options = {
    "context_length": metadata.context_length,
    "prediction_length": metadata.prediction_length,
    "conts": [
        "HUFL",
        "HULL",
        "MUFL",
        "MULL",
        "LUFL",
        "LULL",
    ],
}

trn_dl, val_dl, test_dl = nnts.torch.utils.create_dataloaders_from_split_data(
    split_data,
    Dataset=nnts.torch.datasets.TimeseriesDataset,
    dataset_options=dataset_options,
    Sampler=nnts.torch.datasets.TimeSeriesSampler,
    batch_size=params.batch_size,
    transforms = [nnts.torch.preprocessing.StandardScaler()]
)

# net = nnts.torch.models.TSMixer(metadata.context_length, metadata.prediction_length,7,7, num_blocks=4, ff_dim=256, dropout_rate=0.9)
import nnts.torch.models.dlinear
params = nnts.torch.models.dlinear.get_mutlivariate_params()

net = nnts.torch.models.DLinear(
    metadata.prediction_length,
    metadata.context_length,
    7,
    params
)

AttributeError: module 'nnts.torch.models' has no attribute 'dLinear'

In [134]:
val_dl.dataset.X.shape

torch.Size([1, 2922, 7])

In [135]:
trner = nnts.torch.trainers.ValidationTorchEpochTrainer(net, params, metadata)
evaluator = trner.train(trn_dl, val_dl)
y_hat, y = evaluator.evaluate(
    test_dl, metadata.prediction_length, metadata.context_length
)


TSMixer(
  (mixer_layers): Sequential(
    (0): MixerLayer(
      (time_mixing): TimeMixing(
        (norm): TimeBatchNorm2d(2352, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dropout): Dropout(p=0.9, inplace=False)
        (fc1): Linear(in_features=336, out_features=336, bias=True)
      )
      (feature_mixing): FeatureMixing(
        (norm_before): TimeBatchNorm2d(2352, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (norm_after): Identity()
        (dropout): Dropout(p=0.9, inplace=False)
        (fc1): Linear(in_features=7, out_features=256, bias=True)
        (fc2): Linear(in_features=256, out_features=7, bias=True)
        (projection): Identity()
      )
    )
    (1): MixerLayer(
      (time_mixing): TimeMixing(
        (norm): TimeBatchNorm2d(2352, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dropout): Dropout(p=0.9, inplace=False)
        (fc1): Linear(in_features=336, out_features=336, bias=True

In [136]:
y_hat.shape, y.shape

(torch.Size([2251, 336, 7]), torch.Size([2251, 336, 7]))

In [137]:
nnts.metrics.calculate_seasonal_error(trn_dl, metadata.seasonality)

tensor([[0.26181477]])

In [138]:

test_metrics = nnts.metrics.calc_metrics(
    y_hat, y, nnts.metrics.calculate_seasonal_error(trn_dl, metadata.seasonality)
)
test_metrics

{'mse': 2.187605619430542,
 'abs_error': 5776184.0,
 'abs_target_sum': 7172264.0,
 'abs_target_mean': 1.3547011613845825,
 'seasonal_error': 0.26181477308273315,
 'mean_mase': 4.1671013832092285,
 'mean_mape': 1.209301471710205,
 'mean_smape': 1.298594355583191,
 'mean_msmape': 1.0205368995666504,
 'mean_mae': 1.0910087823867798,
 'mean_rmse': 1.192393183708191,
 'median_mase': 3.2347700595855713,
 'median_smape': 1.339519739151001,
 'median_msmape': 1.0308233499526978,
 'median_mae': 0.8469105958938599,
 'median_rmse': 0.9646993279457092}

In [21]:
0.3049468696117401**2

0.09299259328599963

In [ ]:
nnts.metrics.calc_metrics(y_hat[:, :1, :], y[:, :1, :], nnts.metrics.calculate_seasonal_error(trn_dl, metadata))

In [ ]:
def save_results(y_hat, y, name):
    torch.save(y_hat, f"{PATH}/{name}_y_hat.pt")
    torch.save(y, f"{PATH}/{name}_y.pt")
save_results(y_hat, y, scenario.name)

In [ ]:
covariate_name = f"cov-1-pearsn-0.68-pl-{str(scenario.prediction_length)}-seed-{scenario.seed}"
covariate_y_hat = torch.load(f"{PATH}/{covariate_name}_y_hat.pt")
covariate_y = torch.load(f"{PATH}/{covariate_name}_y.pt")

In [ ]:
def calculate_forecast_horizon_metrics(y_hat, y, metadata, metric="mae"):
    forecast_horizon_metrics = []
    for i in range(1, metadata.prediction_length):
        metrics = nnts.metrics.calc_metrics(y[:, :i, :], y_hat[:, :i, :], metadata.freq, metadata.seasonality)
        forecast_horizon_metrics.append(metrics[metric])
    return forecast_horizon_metrics

forecast_horizon_metrics = calculate_forecast_horizon_metrics(y_hat, y, metadata, "mae")
covariate_forecast_horizon_metrics = calculate_forecast_horizon_metrics(covariate_y_hat, covariate_y, metadata, "mae")

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
plt.plot(forecast_horizon_metrics, label='univariate')
plt.plot(covariate_forecast_horizon_metrics, label='covariate (0.68)')
plt.xlabel("Forecast Horizon")
plt.ylabel("Error (MAE)")
plt.legend()
plt.show()

In [ ]:
csv_aggregator = nnts.datasets.CSVFileAggregator(PATH, "results")
results = csv_aggregator()

In [ ]:
df.tail(metadata.prediction_length*50)['y'].plot()

In [ ]:
y_hat_last = y_hat[:, :1, ...]
y_last = y[:, :1, ...]
df_test = df.tail(y_hat_last.shape[0])
df_test["y_check"] = y_last.squeeze()
df_test["y_hat"] = y_hat_last.squeeze()
df_test[["y", "y_check", "y_hat"]]
df_test.set_index("ds")[["y_check", "y_hat"]].iloc[4500:4500+336].plot(figsize=(20, 10))

In [ ]:
test_metrics = nnts.metrics.calc_metrics(y_last, y_hat_last, metadata.freq, metadata.seasonality)
test_metrics